In [1]:
import requests, random
from requests_ip_rotator import ApiGateway, ALL_REGIONS
from fake_useragent import UserAgent
import pandas as pd
#from tqdm import tqdm
from bs4 import BeautifulSoup
#from selenium.webdriver.chrome.options import Options
#from selenium import webdriver

/home/maaz-lfd/anaconda3/envs/py38/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
df = pd.read_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/round_5_CT_metdata.csv')[['pmcid','doi']]
GS_paper_df = pd.read_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/GS_paper_metadata.csv')

In [3]:
df.head()

,pmcid,doi
0,PMC137267,10.1186/cc987
1,PMC137274,10.1186/cc1013
2,PMC1475929,10.1093/ecam/nel003
3,PMC1481583,10.1186/1465-9921-7-74
4,PMC1525174,10.1186/1471-2458-6-126


In [4]:
len(df)

7802

In [5]:
dois_completed = GS_paper_df['doi'].values.tolist()

In [6]:
dois_completed

['10.1186/cc987',
 '10.1186/cc1013',
 '10.1093/ecam/nel003',
 '10.1186/1465-9921-7-74',
 '10.1186/1471-2458-6-126',
 '10.1186/1476-0711-5-7',
 '10.1186/cc4981',
 '10.1371/journal.pone.0000143',
 '10.1186/cc5944',
 '10.1186/cc5675',
 '10.1093/nar/gkn518',
 '10.1371/journal.pone.0003548',
 '10.1186/1750-1172-3-34',
 '10.1371/journal.pone.0006314',
 '10.1186/cc7875',
 '10.1007/s00281-009-0164-5',
 '10.2174/138161208786898806',
 '10.1186/cc8183',
 '10.1051/vetres/2009071',
 '10.1371/journal.pone.0009332',
 '10.1155/2010/174378',
 '10.3201/eid1310.061262',
 '10.1093/ecam/nep152',
 '10.1371/journal.pone.0011721',
 '10.1203/pdr.0b013e3181e9f3a0',
 '10.1371/journal.pone.0012262',
 '10.1186/1750-1172-5-22',
 '10.1371/journal.pone.0013137',
 '10.1371/journal.pmed.1000346',
 '10.1371/journal.pone.0013047',
 '10.1371/journal.pmed.1000362',
 '10.1007/s00705-010-0768-z',
 '10.1371/journal.pntd.0000892',
 '10.1093/nar/gkq650',
 '10.1371/journal.pone.0015301',
 '10.1155/2010/817580',
 '10.1186/2042-64

In [7]:
def parse_url(url,gateway):
        user_agent = UserAgent()
        browser_list = ['chrome', 'opera', 'firefox', 'safari', 'edge', 'internet explorer']
        browser = random.choice(browser_list)
        # Assign gateway to session
        session = requests.Session()
        session.mount("https://scholar.google.com", gateway)

        # Request to GOOGLE SCHOLAR
        response = session.get(url,headers={'User-Agent':user_agent[browser]})
        print("REQUEST RESPONSE ",response.status_code)

        return BeautifulSoup(response.content, 'html.parser'), int(response.status_code)

In [8]:
def init_proxy():
    return ApiGateway('https://scholar.google.com', regions=ALL_REGIONS)

In [10]:
# Create gateway object and initialise in AWS
gateway = init_proxy()
gateway.start()
for index, row in df.iterrows():
    print("DOI ",row['doi'])
    if row['doi'] not in dois_completed:
        url = f"https://scholar.google.com/scholar?q={str(row['doi'])}"

        #Initialize Proxy and Request to GOOGLE SCHOLAR
        try:
            parsed_response, status_code = parse_url(url=url,gateway=gateway)
            if status_code != 200:
                print("IF1")
                gateway = init_proxy()
                gateway.start()
                parsed_response, status_code = parse_url(url=url,gateway=gateway)
                
        except ConnectionError:
            print("EXCEPT1")
            gateway = init_proxy()
            gateway.start()
            parsed_response, status_code = parse_url(url=url,gateway=gateway)

        div = parsed_response.find('div',{'class':'gs_ri'})
        try:
            paper_url = div.find('h3').find('a').get('href')
        except AttributeError:
            paper_url = ''

        try:
            paper_citation = int(div.find('div',{'class':'gs_fl'}).find_all('a')[2].get_text().split()[-1])
        except ValueError:
            paper_citation = 0
        
        # author_div = div.find('div',{'class':'gs_fma_p'}).find_all('a')
        try:
            author_div = parsed_response.find('div',{'class':'gs_fma_p'}).find_all('a')
        except AttributeError:
            author_div = []
        author_name = []
        author_url = []
        author_citation = 0
        h_index = 0
        print("author_div ",author_div)
        for author in author_div:
            url = f"https://scholar.google.com/{author.get('href')}"

            author_name.append(author.get_text())
            author_url.append(url)
            
            print('url ',url)
            # Initialize Proxy and Request to PAPER AUTHOR GOOGLE SCHOLAR
            parsed_author_response,status_code = parse_url(url=url,gateway=gateway)
            if status_code != 200:
                print("IF2")
                gateway = init_proxy()
                gateway.start()
                parsed_author_response,status_code = parse_url(url=url,gateway=gateway)

            # print(parsed_author_response)
            citation_table = parsed_author_response.find('table',{'id':'gsc_rsb_st'}).find('tbody').find_all('tr')
            author_citation = citation_table[0].find('td',{"class":'gsc_rsb_std'}).get_text()
            h_index = citation_table[1].find('td',{"class":'gsc_rsb_std'}).get_text()

        print("Link ",paper_url)
        print("Paper Citation ",paper_citation)
        print("Author Citation ",author_citation)
        print("H-Index ",h_index)
        print("Author Name ",author_name)
        print("Author URL ",author_url)

        GS_paper_df = GS_paper_df.append({'pmcid':row['pmcid'], 'doi':row['doi'], 'citations':paper_citation, 'author':';'.join(author for author in author_name), 
        'author_link':';'.join(url for url in author_url), 'author_h_index':h_index, 'publisher_link':paper_url, 'author_citations':author_citation}, ignore_index=True)
        GS_paper_df.to_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/GS_paper_metadata.csv',index=False)
        dois_completed.append(row['doi'])

Starting API gateways in 21 regions.
Could not create region (some regions require manual enabling): me-south-1
Could not create region (some regions require manual enabling): ap-east-1
Could not create region (some regions require manual enabling): af-south-1
Could not create region (some regions require manual enabling): eu-south-1
Using 17 endpoints with name 'https://scholar.google.com - IP Rotate API' (0 new).
DOI  10.1186/cc987
DOI  10.1186/cc1013
DOI  10.1093/ecam/nel003
DOI  10.1186/1465-9921-7-74
DOI  10.1186/1471-2458-6-126
DOI  10.1186/1476-0711-5-7
DOI  10.1186/cc4981
DOI  10.1371/journal.pone.0000143
DOI  10.1186/cc5944
DOI  10.1186/cc5675
DOI  10.1093/nar/gkn518
DOI  10.1371/journal.pone.0003548
DOI  10.1186/1750-1172-3-34
DOI  10.1371/journal.pone.0006314
DOI  10.1186/cc7875
DOI  10.1007/s00281-009-0164-5
DOI  10.2174/138161208786898806
DOI  10.1186/cc8183
DOI  10.1051/vetres/2009071
DOI  10.1371/journal.pone.0009332
DOI  10.1155/2010/174378
DOI  10.3201/eid1310.061262
D

In [17]:
parsed_response.get_text()

'Google ScholarLoading...The system can\'t perform the operation now. Try again later.CiteAdvanced searchFind articleswith all of the wordswith the exact phrasewith at least one of the wordswithout the wordswhere my words occuranywhere in the articlein the title of the articleReturn articles authored bye.g., "PJ Hayes" or McCarthyReturn articles published ine.g., J Biol Chem or NatureReturn articles dated between\xa0—\xa0e.g., 1996Saved to My libraryDoneRemove articleArticlesCase lawProfilesMy profileMy libraryAlertsMetricsAdvanced searchSettingsSign inSign inArticlesScholarMy profileMy libraryYearAny timeSince 2023Since 2022Since 2019Sort by relevanceSort by dateAny typeReview articlesinclude patentsinclude citationsAny timeSince 2023Since 2022Since 2019Custom range... — SearchSort by relevanceSort by dateAny typeReview articlesinclude patentsinclude citations [HTML] thieme-connect.com[HTML][HTML] Hypercoagulation and antithrombotic treatment in coronavirus 2019: a new challengeF Viol

In [18]:
parsed_response.find('div',{'class':'gs_fl'})

In [10]:
parsed_response.find('div',{'class':'gs_fl'})

In [152]:
url = f"https://scholar.google.com/scholar?q=10.1186/s40560-019-0415-z"
response, status_code = parse_url(url=url,gateway=gateway)

REQUEST RESPONSE  200


In [153]:
div = response.find('div',{'class':'gs_ri'})
int(div.find('div',{'class':'gs_fl'}).find_all('a')[2].get_text().split()[-1])

21

In [142]:
# response.find('div',{'class':'gs_fl'}).find_all('a')[2].get_text().split()[-1]
a = response.find_all('div',{'class':'gs_fl'})
a

[<div class="gs_ggs gs_fl"><div class="gs_ggsd"><div class="gs_or_ggsm" ontouchstart="gs_evt_dsp(event)" tabindex="-1"><a data-clk="hl=sv&amp;sa=T&amp;oi=gga&amp;ct=gga&amp;cd=0&amp;d=13731880663426737587&amp;ei=GOoJZejTMeGWy9YPrs6WoAU" data-clk-atid="syWbcbVwkb4J" href="https://downloads.hindawi.com/journals/ecam/2006/105453.pdf"><span class="gs_ctg2">[PDF]</span> hindawi.com</a></div></div></div>,
 <div class="gs_fl gs_flb gs_invis"><a class="gs_or_sav gs_or_btn" href="javascript:void(0)" role="button"><svg class="gs_or_svg" viewbox="0 0 15 16"><path d="M7.5 11.57l3.824 2.308-1.015-4.35 3.379-2.926-4.45-.378L7.5 2.122 5.761 6.224l-4.449.378 3.379 2.926-1.015 4.35z"></path></svg><span class="gs_or_btn_lbl">Spara</span></a> <a aria-controls="gs_cit" aria-haspopup="true" class="gs_or_cit gs_or_btn gs_nph" href="javascript:void(0)" role="button"><svg class="gs_or_svg" viewbox="0 0 15 16"><path d="M6.5 3.5H1.5V8.5H3.75L1.75 12.5H4.75L6.5 9V3.5zM13.5 3.5H8.5V8.5H10.75L8.75 12.5H11.75L13.5 

In [148]:
len(a)

2

In [146]:
a[1].find_all('a')

[<a class="gs_or_sav gs_or_btn" href="javascript:void(0)" role="button"><svg class="gs_or_svg" viewbox="0 0 15 16"><path d="M7.5 11.57l3.824 2.308-1.015-4.35 3.379-2.926-4.45-.378L7.5 2.122 5.761 6.224l-4.449.378 3.379 2.926-1.015 4.35z"></path></svg><span class="gs_or_btn_lbl">Spara</span></a>,
 <a aria-controls="gs_cit" aria-haspopup="true" class="gs_or_cit gs_or_btn gs_nph" href="javascript:void(0)" role="button"><svg class="gs_or_svg" viewbox="0 0 15 16"><path d="M6.5 3.5H1.5V8.5H3.75L1.75 12.5H4.75L6.5 9V3.5zM13.5 3.5H8.5V8.5H10.75L8.75 12.5H11.75L13.5 9V3.5z"></path></svg><span>Citera</span></a>,
 <a href="/scholar?cites=13731880663426737587&amp;as_sdt=2005&amp;sciodt=0,5&amp;hl=sv">Citerat av 113</a>,
 <a href="/scholar?q=related:syWbcbVwkb4J:scholar.google.com/&amp;scioq=10.1093/ecam/nel003&amp;hl=sv&amp;as_sdt=0,5">Relaterade artiklar</a>,
 <a class="gs_nph" href="/scholar?cluster=13731880663426737587&amp;hl=sv&amp;as_sdt=0,5">Alla 16 versionerna</a>,
 <a class="gs_or_mor" hre

In [147]:
a[1].find_all('a')[2]

<a href="/scholar?cites=13731880663426737587&amp;as_sdt=2005&amp;sciodt=0,5&amp;hl=sv">Citerat av 113</a>

In [136]:
div.find('h3').find('a').get_text()

'Antibodies to watch in 2020'

In [132]:
div = parsed_response.find('div',{'class':'gs_ri'})

In [133]:
int(div.find('div',{'class':'gs_fl'}).find_all('a')[2].get_text().split()[-1])

396

In [135]:
div.find('div',{'class':'gs_fl'}).find_all('a')

[<a class="gs_or_sav gs_or_btn" href="javascript:void(0)" role="button"><svg class="gs_or_svg" viewbox="0 0 15 16"><path d="M7.5 11.57l3.824 2.308-1.015-4.35 3.379-2.926-4.45-.378L7.5 2.122 5.761 6.224l-4.449.378 3.379 2.926-1.015 4.35z"></path></svg><span class="gs_or_btn_lbl">Save</span></a>,
 <a aria-controls="gs_cit" aria-haspopup="true" class="gs_or_cit gs_or_btn gs_nph" href="javascript:void(0)" role="button"><svg class="gs_or_svg" viewbox="0 0 15 16"><path d="M6.5 3.5H1.5V8.5H3.75L1.75 12.5H4.75L6.5 9V3.5zM13.5 3.5H8.5V8.5H10.75L8.75 12.5H11.75L13.5 9V3.5z"></path></svg><span>Cite</span></a>,
 <a href="/scholar?cites=3466683180027005099&amp;as_sdt=2005&amp;sciodt=0,5&amp;hl=en">Cited by 396</a>,
 <a href="/scholar?q=related:q8hUJvwhHDAJ:scholar.google.com/&amp;scioq=10.1080/19420862.2019.1703531&amp;hl=en&amp;as_sdt=0,5">Related articles</a>,
 <a class="gs_nph" href="/scholar?cluster=3466683180027005099&amp;hl=en&amp;as_sdt=0,5">All 7 versions</a>,
 <a class="gs_or_mor gs_oph" h

In [127]:
div

<div class="gs_ri"><h3 class="gs_rt" ontouchstart="gs_evt_dsp(event)"><a data-clk="hl=en&amp;sa=T&amp;ct=res&amp;cd=0&amp;d=3466683180027005099&amp;ei=TNcJZd7FIYLyyASFl7bwBQ" data-clk-atid="q8hUJvwhHDAJ" href="https://www.tandfonline.com/doi/abs/10.1080/19420862.2019.1703531" id="q8hUJvwhHDAJ">Antibodies to watch in 2020</a></h3><div class="gs_a gs_fma_s">H Kaplon, M Muralidharan, <a href="/citations?user=PivGTYYAAAAJ&amp;hl=en&amp;oi=sra">Z Schneider</a>, <a href="/citations?user=EGukXJcAAAAJ&amp;hl=en&amp;oi=sra">JM Reichert</a> - MAbs, 2020 - Taylor &amp; Francis</div><div class="gs_a gs_fma_p"><div class="gs_fmaa">H Kaplon, M Muralidharan, <a href="/citations?user=PivGTYYAAAAJ&amp;hl=en&amp;oi=sra">Z Schneider</a>, <a href="/citations?user=EGukXJcAAAAJ&amp;hl=en&amp;oi=sra">JM Reichert</a></div>MAbs, 2020<span class="gs_pdot">•</span>Taylor &amp; Francis</div><div class="gs_rs gs_fma_s">This 2020 installment of the annual 'Antibodies to Watch'series documents the antibody <br/>ther

In [125]:
response.find('div',{'class':'gs_fma_p'}).find_all('a')[0].get('href')

'/citations?user=K26N3KEAAAAJ&hl=en&oi=sra'

In [ ]:
ALL_REGIONS

['us-east-1',
 'us-east-2',
 'us-west-1',
 'us-west-2',
 'eu-west-1',
 'eu-west-2',
 'eu-west-3',
 'eu-north-1',
 'eu-central-1',
 'ca-central-1',
 'ap-south-1',
 'ap-northeast-3',
 'ap-northeast-2',
 'ap-southeast-1',
 'ap-southeast-2',
 'ap-northeast-1',
 'sa-east-1',
 'ap-east-1',
 'af-south-1',
 'eu-south-1',
 'me-south-1']

In [ ]:
DEFAULT_REGIONS

['us-east-1',
 'us-east-2',
 'us-west-1',
 'us-west-2',
 'eu-west-1',
 'eu-west-2',
 'eu-west-3',
 'eu-north-1',
 'eu-central-1',
 'ca-central-1']

In [ ]:
EXTRA_REGIONS

['us-east-1',
 'us-east-2',
 'us-west-1',
 'us-west-2',
 'eu-west-1',
 'eu-west-2',
 'eu-west-3',
 'eu-north-1',
 'eu-central-1',
 'ca-central-1',
 'ap-south-1',
 'ap-northeast-3',
 'ap-northeast-2',
 'ap-southeast-1',
 'ap-southeast-2',
 'ap-northeast-1',
 'sa-east-1']

In [44]:
gateway = init_proxy()
gateway.start()

Starting API gateways in 10 regions.
Using 10 endpoints with name 'https://scholar.google.com - IP Rotate API' (0 new).


['0as80273i1.execute-api.eu-west-3.amazonaws.com',
 '09zn644dob.execute-api.us-east-1.amazonaws.com',
 '01vrh3v9ki.execute-api.ca-central-1.amazonaws.com',
 '007q7s392l.execute-api.us-west-2.amazonaws.com',
 '343nl230c5.execute-api.us-west-1.amazonaws.com',
 '0llffx12db.execute-api.eu-north-1.amazonaws.com',
 '0fywgub9nd.execute-api.eu-central-1.amazonaws.com',
 '00xxrve1q3.execute-api.us-east-2.amazonaws.com',
 '027opvdsn2.execute-api.eu-west-2.amazonaws.com',
 '02lsfk84b9.execute-api.eu-west-1.amazonaws.com']